# fazer a imagem ser georeferenciada e salvar na pasta data/raw

In [18]:
!pip install rasterio


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [20]:
import rasterio
from rasterio.transform import from_bounds
from rasterio.crs import CRS
import numpy as np

In [21]:
# Your input image (the satellite export)
input_path = "data/raw/satellite_uncropped.tif"  # change if it’s PNG
output_path = "data/raw/satellite_georeferenced.tif"

In [22]:
# Image bounds and dimensions
bounds = (693975.1349, 7467497.9736, 694193.4778, 7467672.6713)  # (left, bottom, right, top)
width = 1088
height = 630
crs = CRS.from_epsg(31983)
transform = from_bounds(*bounds, width=width, height=height)

In [23]:
# Read original image
with rasterio.open(input_path) as src:
    data = src.read()

In [24]:
# Save as GeoTIFF with georeferencing
with rasterio.open(
    output_path, 'w',
    driver='GTiff',
    height=height,
    width=width,
    count=data.shape[0],  # number of bands
    dtype=data.dtype,
    crs=crs,
    transform=transform
) as dst:
    dst.write(data)

print("✅ Done! GeoTIFF saved as:", output_path)


✅ Done! GeoTIFF saved as: data/raw/satellite_georeferenced.tif


# salvar nos downloads

In [25]:
import os

In [26]:
output_path_downloads = os.path.expanduser("~/Downloads/satellite_georeferenced.tif")

In [27]:
with rasterio.open(input_path) as src:
    data = src.read()
    dtype = src.dtypes[0]
    count = src.count

In [28]:
with rasterio.open(
    output_path_downloads, 'w',
    driver='GTiff',
    height=height,
    width=width,
    count=count,
    dtype=dtype,
    crs=crs,
    transform=transform
) as dst:
    dst.write(data)

print("✅ Saved to Finder Downloads folder:", output_path)

✅ Saved to Finder Downloads folder: data/raw/satellite_georeferenced.tif


# mudar as dimensoes da simulação

In [30]:
!pip install Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 8.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [31]:
import rasterio
from rasterio.enums import Resampling
from PIL import Image
import numpy as np

In [32]:
src_path = "data/raw/satellite_georeferenced.tif"
dst_path = "data/processed/satellite_bg.png"

In [33]:
with rasterio.open(src_path) as src:
    data = src.read(
        out_shape=(src.count, 90, 110),           # bands, rows, cols
        resampling=Resampling.bilinear
    )
    # transpose to H×W×bands, scale 0-255 and save
    arr = np.moveaxis(data, 0, -1)
    img = Image.fromarray(arr[:, :, :3].astype("uint8"))  # keep RGB
    img.save(dst_path)